Task 1:
Complete the following steps to establish a connection pool:

To create a connection pool, import MySQLConnectionPool class from MySQL Connector/Python.

To find the information on the error, import the Error class from MySQL Connector/Python.

Define your database configurations as a Python dictionary object called dbconfig.

Establish a connection pool [pool_name = pool_b] with two connections. 

Implement error handling using a try-except block in case the connection fails. 

In [2]:
#Database configuration
dbconfig = {
    "host":"tascloud",
    "port":3306,
    "user":"test",
    "password":"1-TestPassword",
    "database":"little_lemon_db"
}

In [3]:
from mysql.connector import pooling, Error

try:
    cpool = pooling.MySQLConnectionPool(pool_name="pool_b",
                                                  pool_size=3,
                                                  pool_reset_session=True,
                                                  **dbconfig)
except Error as e:
    print("Could not create a connection pool. See error for details")
    print(e.args)

Task 2:
Three guests are trying to book dinner slots simultaneously. Get the connections from pool_b and insert the following data in the Bookings table:
TIP: You need to add a connection to connect the third guest.


Guest 1:

Table Number: 8

First Name: Anees

Last Name: Java

Booking Time: 18:00

EmployeeID: 6


Guest 2:

Table Number: 5

First Name: Bald

Last Name: Vin

Booking Time: 19:00

EmployeeID: 6


Guest 3:

Table Number: 12

First Name: Jay

Last Name: Kon

Booking Time: 19:30 

EmployeeID: 6



In [4]:
query1 = """
INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
VALUES
(8, 'Anees', 'Java', '18:00:00', 6);
"""

query2 = """
INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
VALUES
(5, 'Bald', 'Vin', '19:00:00', 6);
"""

query3 = """
INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
VALUES
(12, 'Jay', 'Kon', '19:30:00', 6);
"""

In [6]:
# Creating procedures
# Q1
con1 = cpool.get_connection()
cmd1 = con1.cursor()
try:
    cmd1.execute(query1)
    con1.commit()
except Exception as err:
    print(err, "Query 1")
finally:
    con1.close()

# Q2
con2 = cpool.get_connection()
cmd2 = con2.cursor()
try:
    cmd2.execute(query2)
    con2.commit()
except Exception as err:
    print(err, "Query 2")
finally:
    con2.close()

# Q3
con1 = cpool.get_connection()
cmd1 = con1.cursor()
try:
    cmd1.execute(query3)
    con1.commit()
except Exception as err:
    print(err, "Query 3")
finally:
    con1.close()


Task 3:
Create a report containing the following information:

The name and EmployeeID of the Little Lemon manager.

The name and role of the employee who receives the highest salary.

The number of guests booked between 18:00 and 20:00.

The full name and BookingID of all guests waiting to be seated with the receptionist in sorted order with respect to their BookingSlot.

In [7]:
query1 = "SELECT EmployeeID, Name FROM Employees WHERE Role = 'Manager';"
query2 = "SELECT Name, Role FROM Employees ORDER BY Annual_Salary DESC limit 1;"
query3 = "SELECT COUNT(*) AS NumberofGuests FROM Bookings WHERE BookingSlot >= '18:00:00' AND BookingSlot <= '20:00:00';"
query4 = """SELECT B.BookingID, concat(B.GuestFirstName, ' ', B.GuestLastName) AS GuestFullName 
FROM Bookings AS B
INNER JOIN Employees AS E
	ON B.EmployeeID = E.EmployeeID
WHERE E.Role = 'Receptionist'
ORDER BY B.BookingSlot;
"""

In [10]:
def printReport(query, description):
    # Running report
    print(description)
    con1 = cpool.get_connection()
    cmd1 = con1.cursor()
    cmd1.execute(query)
    dataset = cmd1.fetchall()
    column_names = []
    #getting column names
    for result in cmd1.description:
        column_names.append(result[0])
    print(column_names)
    for data in dataset:
        print(data)
    con1.close()

# Running report
printReport(query1, "The name and EmployeeID of the Little Lemon manager:")
printReport(query2, "The name and role of the employee who receives the highest salary.")
printReport(query3, "The number of guests booked between 18:00 and 20:00.")
printReport(query4, "The full name and BookingID of all guests waiting to be seated with the receptionist in sorted order with respect to their BookingSlot.")


The name and EmployeeID of the Little Lemon manager:
['EmployeeID', 'Name']
(1, 'Mario Gollini')
The name and role of the employee who receives the highest salary.
['Name', 'Role']
('Mario Gollini', 'Manager')
The number of guests booked between 18:00 and 20:00.
['NumberofGuests']
(8,)
The full name and BookingID of all guests waiting to be seated with the receptionist in sorted order with respect to their BookingSlot.
['BookingID', 'GuestFullName']
(7, 'Anees Java')
(11, 'Anees Java')
(12, 'Bald Vin')
(13, 'Jay Kon')


Task 4:
Create a stored procedure named BasicSalesReport that returns the following statistics: 

Total sales

Average sale

Minimum bill paid

Maximum bill paid

In [11]:
# I don't know what the question means by sales??? I'll use total and average bill instead
proc_basic_sales = """
CREATE PROCEDURE BasicSalesReport()
SELECT SUM(BillAmount) as TotalSales, AVG(BillAmount) as AverageSale, MAX(BillAmount) as MaxBill, MIN(BillAmount) as MinBill FROM Orders;
"""

con1 = cpool.get_connection()
cmd1 = con1.cursor()
try:
    cmd1.execute(proc_basic_sales)
except Exception as err:
    print(err, "BasicSalesReport")


# Running the proc
cmd1.callproc("BasicSalesReport")
results = next( cmd1.stored_results() )
dataset = results.fetchall()
column_names = []
#getting column names
for result in results.description:
    column_names.append(result[0])
print(column_names)
for data in dataset:
    print(data)
con1.close()

['TotalSales', 'AverageSale', 'MaxBill', 'MinBill']
(Decimal('243'), Decimal('48.6000'), 86, 37)


Task 5:
Little Lemon needs to display the next three upcoming bookings from the Bookings table on the kitchen screen to notify their chefs which orders are due next. To complete this task, carry out the following steps:

Get a connection from the pool.

Create a buffered cursor.

Combine the data from the Bookings and the Employee tables. Sort the retrieved records in ascending order. Then display the information of the first three guests. 

Returned the connection back to the pool.


The output should be as follows:

[BookingSlot]

[Guest_name]

[Assigned to: Employee Name [Employee Role]]

In [16]:
query = """SELECT 
	B.BookingSlot, 
    concat(B.GuestFirstName, ' ', B.GuestLastName) as Guest_name, 
    concat("Assigned to: ", E.Name, ' [' , E.Role, "]") as AssignedTo
FROM Bookings as B
INNER JOIN Employees as E;
	On B.EmployeeID = E.EmployeeID
ORDER BY BookingSlot
limit 3;"""

con1 = cpool.get_connection()
cmd1 = con1.cursor(buffered=True)
cmd1.execute(query)
dataset = cmd1.fetchmany(3)
column_names = []
#getting column names
for result in cmd1.description:
    column_names.append(result[0])
print(column_names)

for data in dataset:
    print(data)
cmd1.close()

['BookingSlot', 'Guest_name', 'AssignedTo']
(datetime.timedelta(seconds=68400), 'Anna Iversen', 'Assigned to: Mario Gollini [Manager]')
(datetime.timedelta(seconds=68400), 'Anna Iversen', 'Assigned to: Adrian Gollini [Assistant Manager]')
(datetime.timedelta(seconds=68400), 'Anna Iversen', 'Assigned to: Giorgos Dioudis [Head Chef]')


True